In [1]:
import torch
from train import train
from data_module import EEGDataModule
from model import ViTransformer, LSTM, ConvLSTM, DeepConvNet, RNN, LSTM, ShallowConvNet
from ATCNet import ATCNet

/home/danny/miniconda3/envs/rl/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
device = torch.device(0)

In [4]:
from preprocessing import mean_max_subsampling

In [5]:
train_params = {
    "train_epochs": 10,
    "accumulate_grad_batches": 1,
    "test_size": 0.2,
    "random_state": 42,
    "data_dir": "../../Data/",
    "train_batch_size": 64,
    "eval_batch_size": 32
}

In [6]:
dataset = EEGDataModule(args=train_params)

In [7]:
dataset.setup(transform=None)

INFO:data_module:Training data shape: (1692, 22, 1000)
INFO:data_module:Training labels shape: (1692,)


In [8]:
train_dataloader, valid_dataloader = dataset.train_dataloader(), dataset.val_dataloader()

INFO:data_module:loaded 1692 train data instances
INFO:data_module:loaded 423 train data instances


In [9]:
model = ATCNet()

/home/danny/Desktop/ECE-147-Project/EEG/src/ATCNet.py:240: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  torch.nn.init.kaiming_uniform(conv.weight)
/home/danny/Desktop/ECE-147-Project/EEG/src/ATCNet.py:242: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  torch.nn.init.kaiming_uniform(conv.weight)
/home/danny/miniconda3/envs/rl/lib/python3.7/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [10]:
from torchinfo import summary
# Only uses outputs of modules.
print(summary(model))

Layer (type:depth-idx)                                  Param #
ATCNet                                                  --
├─Convolution_module: 1-1                               --
│    └─Conv2d: 2-1                                      1,024
│    └─BatchNorm2d: 2-2                                 32
│    └─Conv2d: 2-3                                      704
│    └─BatchNorm2d: 2-4                                 64
│    └─ELU: 2-5                                         --
│    └─AvgPool2d: 2-6                                   --
│    └─AvgPool2d: 2-7                                   --
│    └─Dropout: 2-8                                     --
│    └─Dropout: 2-9                                     --
│    └─Conv2d: 2-10                                     16,384
│    └─BatchNorm2d: 2-11                                64
├─ModuleList: 1-2                                       --
│    └─MultiheadAttention: 2-12                         3,168
│    │    └─NonDynamicallyQuantizableLin

In [11]:
loss_hist, acc_hist, val_loss_hist, val_acc_hist = train(model, train_dataloader, valid_dataloader, device, weight_decay=1e-5)

  0%|                              | 0/100 [00:00<?, ?it/s]/home/danny/miniconda3/envs/rl/lib/python3.7/site-packages/torch/nn/modules/conv.py:460: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /opt/conda/conda-bld/pytorch_1666642814471/work/aten/src/ATen/native/Convolution.cpp:895.)
  self.padding, self.dilation, self.groups)
100%|██████████████████████████████| 100/100 [01:20<00:00,  1.24it/s, acc=0.974, val_acc=0.655]


In [12]:
import numpy as np
val_loss_hist[np.argmin(val_loss_hist)]

0.6469507217407227

In [13]:
acc_hist[99]

0.9739952718676123

In [14]:
print(np.argmin(loss_hist))

90


In [15]:
loss_hist[81]

0.21018585562705994